## 1. 准备环境

### 1.1 安装依赖

现在，让我们安装一些额外的库，例如 langchain 和 python-dotenv。

前者为我们提供了一个构建基于LLM的应用程序的模块化框架，而后者在为在线LLM服务设置API密钥方面为我们节省了时间（有关详细信息，请参见下一节）。

In [1]:
# %%capture
# Install langchain, the library we will learn during our courses
!pip install langchain==0.0.338 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [43]:
%%capture
# Install dotenv, auto-load environment variables from `.env` files
!pip install python-dotenv==1.0.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

此外，让我们安装用于对内容进行标记化和存储在向量数据库上的库，即 tiktoken 和 faiss-cpu。

In [5]:
!pip install poetry -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [50]:
#%%capture
# Install tiktoken, the library used by OpenAI models for tokenizing text strings
!pip install tiktoken -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/bd/ef/91777d3310589c55da4bf0fafa10fdc8ddefa30aa7dfa67b2fc8825bc1f1/tiktoken-0.5.1.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d3/10/6f2d5f8635d7714ad97ce6ade7a643358c4f3e45cde4ed12b7150734a8f3/regex-2023.10.3-cp312-cp312-win_amd64.whl (268 kB)
Failed to build tiktoken


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [38 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\core.py -> build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\load.py -> build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\model.py -> build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\registry.py -> build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\_educational.py -> build\lib.win-amd64-cpython-312\tiktoken
  copying tiktoken\__init__.py -> build\lib.win-amd64-cpython-312\tiktoken
  creating build\lib.win-amd64-cpython-312\tiktoken_ext
  copying tiktoken_ext\openai_public.py -> build\lib.win-amd64-cpython-312\tiktoken_ext
  running egg_info
  writing tiktoken.egg-info\PKG-INFO
  writing dependency_links to tiktoken.egg-info\dependency_links.txt
  writing requirements to tiktoken.eg

In [45]:
%%capture
# Install faiss-cpu, a vector database for storing content along with embedding vectors
!pip install faiss-cpu==1.7.4 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [46]:
%%capture
# Install wikipedia, the library for accessing wikipedia service in code
!pip install wikipedia==1.4.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

然后，安装一些用于访问外部服务的库，例如 wikipedia。

In [47]:
%%capture
# Install wikipedia, the library for accessing wikipedia service in code
!pip install wikipedia==1.4.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

最后，为了测试安装和API密钥的有效性，我们还安装相应供应商的SDK库（即OpenAI和智谱AI）。

In [48]:
%%capture
# Install openai, official SDK by OpenAI for invoking GPT models
!pip install openai==1.3.3 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [49]:
%%capture
# Install zhipu, official SDK by OpenAI for invoking ChatGLM models
!pip install zhipuai==1.0.7 -i https://pypi.tuna.tsinghua.edu.cn/simple

### 1.2 环境变量

In [12]:
# Example `.env` config file.
ECHO ZHIPUAI_API_KEY=<3fb52888d94d1ff2b72ac95bfd269a9e.BVSfK2CHbEb8sU9u > .env

SyntaxError: invalid decimal literal (2605503247.py, line 2)

In [19]:
# Load ZHIPUAI API key from `.env` file.
from dotenv import load_dotenv
load_dotenv()

True

### 1.3 测试准备是否成功

In [20]:
# Test zhipuai installation
import os
import zhipuai

zhipuai.api_key = os.getenv('ZHIPUAI_API_KEY')  # Set API key from envrionment variable

prompt = """You will be provided with a sentence in English, and your task is to translate it into Chinese.

My name is Jane. What is yours?
"""

completion = zhipuai.model_api.invoke(
    model='chatglm_turbo',
    prompt=[
        {'role': 'user', 'content': prompt}
    ],
    temperature=0.,
)

print(completion['data']['choices'][0]['content'])

" 我的名字是简。你的名字叫什么？"


## 2. Langchain基础练习（基于智谱LLM）

与OpenAI不同，LangChain并不原生支持智谱AI的在线LLM服务。相反，我们可以编写一个包装类来将智谱AI的ChatGLP服务移植到LangChain，这要归功于LangChain的模块化接口。这应该类似于我们使用OpenAI的GPT服务时的感觉。

### 2.1 检查ZhipuAI wrapper是否存在

In [21]:
# Check ZhipuAI wrapper existence
!ls -la | grep "zhipuai"

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


### 2.2 简单使用例子

In [26]:
from zhipuai_llm import ZhipuAILLM

prompt = """You will be provided with a sentence in English, and your task is to translate it into Chinese.

My name is Jane. What is yours?
"""

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)

response = llm.predict(prompt)

print(response)

" 我叫简。你叫什么名字？"


#### 练习1 - "计算时间复杂度"

> 💪 Practice yourself.
> Please finish the code for this task, with the following prompt example:
>
> ---------------------------
> 
> ```
> You will be provided with Python code, and your task is to calculate its time complexity.
>
> def foo(n, k):
>    accum = 0
>    for i in range(n):
>        for l in range(k):
>            accum += i
>    return accum
> ```
> 
> ---------------------------
> Try to change the Python code for analysis and see how LLM responses.

In [27]:
from zhipuai_llm import ZhipuAILLM

prompt = """You will be provided with Python code, and your task is to calculate its time complexity.

def foo(n, k):
   accum = 0
   for i in range(n):
       for l in range(k):
           accum += i
   return accum
   respond in chinese
"""

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)

response = llm.predict(prompt)

print(response)

" 这段Python代码的时间复杂度为 O(n * k)，其中n是外部循环的次数，k是内部循环的次数。在这个例子中，内部循环的次数k等于n，所以时间复杂度可以简化为O(n^2)。这意味着当n增大时，函数执行时间将以n的平方速度增加。"


#### 练习2 - “微博情感分析”

> 💪 Practice yourself.
> Please finish the code for this task, with the following prompt example:
>
> ---------------------------
> ```
> You will be provided with a tweet, and your task is to classify its sentiment as 
> positive, neutral, or negative.
> 
> I loved the new Batman movie!
> ```
>
> ---------------------------
> Try to change the tweet text for analysis and see how LLM responses.

In [29]:
from zhipuai_llm import ZhipuAILLM

prompt = """You will be provided with a tweet, and your task is to classify its sentiment as 
positive, neutral, or negative.

I loved the new Batman movie!
   respond answer in chinese
"""

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)

response = llm.predict(prompt)

print(response)

" 正面"


#### 练习3 - “机场代号提取”

> 💪 Practice yourself.
> Please finish the code for this task, with the following prompt example:
>
> ---------------------------
> ```
> You will be provided with a text, and your task is to extract the airport codes from it.
> 
> I want to fly from Orlando to Boston
> ```
>
> ---------------------------
> Try to change the city names and see how LLM responses.

In [30]:
from zhipuai_llm import ZhipuAILLM

prompt = """You will be provided with a text, and your task is to extract the airport codes from it.

I want to fly from Orlando to Boston
   respond answer in chinese
"""

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)

response = llm.predict(prompt)

print(response)

" 从文本中提取的机场代码为：MCO（奥兰多机场代码）和 BOS（波士顿机场代码）。"


### 2.3 探索LLM局限

In [49]:
from zhipuai_llm import ZhipuAILLM

prompt = """Which team won the 1986 FIFA World Cup? respond answer in chinese"""
llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)
response = llm.predict(prompt)
print(f'- 1st response: {response}')

prompt = """Which team won the 2022 FIFA World Cup?"""
llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)
response = llm.predict(prompt)
print(f'- 2nd response: {response}')

[llm/start] [1:llm:ZhipuAILLM] Entering LLM run with input:
{
  "prompts": [
    "Which team won the 1986 FIFA World Cup? respond answer in chinese"
  ]
}
[llm/end] [1:llm:ZhipuAILLM] [3.92s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\" 1986年FIFA世界杯的冠军是阿根廷队。中文回答：阿根廷队赢得了1986年的FIFA世界杯。\"",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
- 1st response: " 1986年FIFA世界杯的冠军是阿根廷队。中文回答：阿根廷队赢得了1986年的FIFA世界杯。"
[llm/start] [1:llm:ZhipuAILLM] Entering LLM run with input:
{
  "prompts": [
    "Which team won the 2022 FIFA World Cup?"
  ]
}
[llm/end] [1:llm:ZhipuAILLM] [3.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\" As an AI language model, I cannot predict future events. The 2022 FIFA World Cup will take place in Qatar, but the winner is yet to be determined as the tournament has not occurred. Stay tuned for updates!\"",
        "generation_info":

In [34]:
from zhipuai_llm import ZhipuAILLM

prompt = """Sum 4829 and 2930, and then multiply by 1923.respond answer in chinese"""

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.)
response = llm.predict(prompt)

print(f'- gpt: {response}')
print(f'- truth:\n\n {(4829 + 2930) * 1923}')

- gpt: " 首先将4829和2930相加，得到7759。然后将7759乘以1923，结果为146,802,247。所以，(4829+2930)×1923=146,802,247。"
- truth:

 14920557


### 2.4 探索Langchain模块化组件设计

📌 打开调试和详细模式

如果您是初学者，我们建议您在LangChain中打开调试和详细模式，在LLM应用程序执行过程中显示中间步骤的额外信息。
查看提示如何填充以及中间LLM生成的响应是个好主意（在正常模式下不应打印任何输出）。

In [36]:
import langchain

langchain.debug = True
langchain.verbose = True

#### Model I/O

In [37]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

from zhipuai_llm import ZhipuAILLM

# [1] Custom output parser, split comma separated strings and return as list
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

# [2] System message template, declare task requirement as prompt
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

# [3] Human message template, here we use Python format string syntax
# (https://docs.python.org/3/library/string.html#formatstrings)
human_template = '{text}'

# [4] We send both messages to LLM for response
chat_prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template),
])

# [5] Build up simple chain with LangChain Expression Language
# (https://python.langchain.com/docs/expression_language/)
chain = chat_prompt | ZhipuAILLM(model='chatglm_turbo') | CommaSeparatedListOutputParser()

# [6] Call simple chain with human input, i.e., text = "colors"
chain.invoke({'text': 'colors'})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "text": "colors"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "text": "colors"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful assistant who generates comma separated lists.\nA user will pass in a category, and you should generate 5 objects in that category in a comma separated list.\nONLY return a comma separated list, and nothing more.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 

['" red', 'blue', 'green', 'yellow', 'purple"']

#### Chains

在接下来的部分，我们将专注于传统的Chain接口。首先开始重写前一节中的ICEL风格链。

In [25]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts.chat import ChatPromptTemplate

from zhipuai_llm import ZhipuAILLM

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

human_template = '{text}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_template),
])

# Equivalent to `chain = chat_prompt | ZhipuAILLM(model='chatglm_turbo') | CommaSeparatedListOutputParser()`
chain = LLMChain(
    llm=ZhipuAILLM(model='chatglm_turbo'),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser(),
)

chain.invoke({'text': 'colors'})

{'text': ['" red', 'blue', 'green', 'yellow', 'purple"']}

然后，让我们看一个更复杂的链。我们将介绍一个简单的两阶段连续链，其中：

1. 为一家制造某种产品的公司提出名称
2. 为提出的公司写一个简短的描述（即口号）

In [42]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts.chat import ChatPromptTemplate

from zhipuai_llm import ZhipuAILLM

product = 'Pure Milk'

# [0] The same LLM instance shared by both chains (remember LLM is stateless)
llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.7)

# [1] Build name chain (1st chain)
name_template = """What is the best name to describe a company that makes {product}? answer in chinese"""
name_prompt = ChatPromptTemplate.from_template(name_template)
name_chain = LLMChain(llm=llm, prompt=name_prompt)

# [2] Build slogan chain (2nd chain)
slogan_template = """Write a 20 words slogan for the following company:{company_name} answer in chinese"""
slogan_prompt = ChatPromptTemplate.from_template(slogan_template)
slogan_chain = LLMChain(llm=llm, prompt=slogan_prompt)

# [3] Construct final chain in a sequencial manner
overall_chain = SimpleSequentialChain(chains=[name_chain, slogan_chain])

# [4] Call our final chain to propose and write slogan
overall_chain.run(product)

[chain/start] [1:chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "Pure Milk"
}
[chain/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "product": "Pure Milk"
}
[llm/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:ZhipuAILLM] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the best name to describe a company that makes Pure Milk? answer in chinese"
  ]
}
[llm/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:ZhipuAILLM] [3.29s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\" 纯净奶制品公司（Pure Milk Products Company）\"",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] [3.29s] Exiting Chain run with output:
{
  "text": "\" 纯净奶制品公司（Pure Milk Products Company）\""
}
[chain/start] [1:chain:SimpleSequentialChain > 4:c

'" 纯净之源，健康守护，奶制品公司的标语：“纯净奶源，品质传承”。"'

#### Memory

回顾一下我们说过的LLM本质上是无状态的，即后续调用永远不会回忆起在之前的调用中提到的信息。让我们看一个例子来说明这个说法。

In [43]:
from zhipuai_llm import ZhipuAILLM 

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.7)
print(f'Initial message: {llm.predict("Hello, my name is Charles.")}')
print(f'Follow-up message: {llm.predict("Well, what is my name?")}')

[llm/start] [1:llm:ZhipuAILLM] Entering LLM run with input:
{
  "prompts": [
    "Hello, my name is Charles."
  ]
}
[llm/end] [1:llm:ZhipuAILLM] [2.81s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\" Hello Charles! It's nice to meet you. How can I help you today? If you have any questions or need assistance, feel free to ask.\"",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
Initial message: " Hello Charles! It's nice to meet you. How can I help you today? If you have any questions or need assistance, feel free to ask."
[llm/start] [1:llm:ZhipuAILLM] Entering LLM run with input:
{
  "prompts": [
    "Well, what is my name?"
  ]
}
[llm/end] [1:llm:ZhipuAILLM] [3.77s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\" As an AI language model, I don't have real-time awareness or knowledge of your name. However, I can help you answer questions about

现在，让我们看看如何在LangChain中为一个对话应用程序添加一个记忆模块。具体来说，我们将使用ConversationBufferMemory记忆模块。

In [28]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from zhipuai_llm import ZhipuAILLM

# [1] Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

# [2] Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key='chat_history')

llm = ZhipuAILLM(model='chatglm_turbo', temperature=0.7)

# [3] Memory should work with Chain for effect
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

print(f'Initial message: {chain.invoke("Hello, my name is Charles.")["text"]}')
print(f'Follow-up message: {chain.invoke("Well, what is my name?")["text"]}')

Initial message: " Hello, Charles! It\\'s a pleasure to meet you. How can I help you today? Is there a specific topic you would like to discuss or do you just want to chat? I\\'m here to assist you in any way I can.\n"
Follow-up message: " \"Charles, your name is Charles.\\n\"\n\nIf you have any other questions or would like to chat about something else, please feel free to ask.\\n\""


#### Retrieval

现在，让我们看一个简单的检索方式，即基于向量存储的检索器，并看看它在LangChain组件中的工作原理。

In [48]:
from dotenv import load_dotenv

load_dotenv()

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from zhipuai_embedding import ZhipuAIEmbeddings

# [1] Load content from disk file
loader = TextLoader('流浪地球.txt')
documents = loader.load()

# [2] Transform file content into splits for storage and retrieve
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# [3] Here we invoke embedding functions provided by OpenAI services, which maps text
#     string of any size into a fixed size embedding vector, where similar text are
#     mapped into vectors of short distance
# [4] We use FAISS as our vector store backend to save content along with embedding vectors
embeddings = ZhipuAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

# [5] Retriever can be directly accessed from vector store instance
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("流浪地球计划")

# [6] Interate around retrieved documents and print first 100 characters of each
for i, doc in enumerate(docs):
    print(f'doc #{i}: {doc.page_content[:100]}...')

RuntimeError: Error loading 流浪地球.txt

### 2.5 LangChain: Hands-On 练习4

在本节中，我们将借助LangChain框架构建一个简单的LLM应用程序。我们即将构建的应用程序是一个文档聊天机器人，允许您就文档文件的内容提出问题。有关更多信息，请参阅[Chatbot](https://python.langchain.com/docs/use_cases/chatbots)。

**step1:**

让我们首先定义要使用的LLM模型。与以前一样，可以使用智谱AI。

In [ ]:
# Write your code here.

**step2:**
  
然后，创建一个用于存储历史聊天消息的记忆，这使得聊天机器人能够记住先前的对话。在这里，不再使用之前的ConversationBufferMemory，而是尝试另一种记忆，即ConversationSummaryMemory。

In [ ]:
# Write your code here.

注意，ConversationSummaryMemory接受一个名为llm的参数。

实际上，这个记忆保留了两种类型的历史对话信息，即历史消息的列表和历史消息的简短摘要。

与ConversationBufferMemory相比，摘要的使用使我们不会使LLM上下文窗口（令牌限制）变得臃肿。

**step3:**

之后，让我们完成检索器部分，即加载文档、拆分文本、转换为嵌入并存储在数据库中。
  
与之前一样，我们将使用FAISS向量存储。

In [ ]:
# load数据资源
# Write your code here
blog_url = 'https://lilianweng.github.io/posts/2023-06-23-agent/'

In [ ]:
# 拆分数据成块
# Write your code here

In [ ]:
# 向量处理存入向量数据库
# Write your code here

**step4:**
最后，让我们将上述组件组合成一个单一的链。我们使用的链是`ConversationalRetrievalChain`。该链的工作方式如下：

1. 使用聊天历史和新问题创建一个“独立问题”。
2. 将这个新问题传递给检索器，并返回相关文档。
3. 将检索到的文档与新问题（默认行为）或原始问题和聊天历史一起传递给LLM，生成最终的响应。

In [ ]:
# Write your code here.

**step5:**
  
现在，让我们测试一下我们的聊天机器人。

In [ ]:
# Question One: 'How do agents use Task decomposition?'
# Write your code here.

In [ ]:
# Question Two: 'What are the various ways to implement memory to support it?'
# Write your code here.

## 3. 基于LLM的Agent（基于OpenAI）

**Agent: Hands-On**
 
Agents的核心思想是使用语言模型选择一系列要执行的动作。

而在Chains中，一系列动作是硬编码的（在代码中）

在Agent中，语言模型被用作推理引擎，确定要执行哪些动作以及顺序。

为了支持构建基于LLM的Agent），LangChain提供了以下模块化组件，即

* `Tool`：包装了一个Python函数和相应的文本描述，它赋予Agent调用外部工具的能力，例如计算器、Python解释器、搜索引擎API。
* `Agent`：扩展了普通的LangChain`Chain`模块，具有一组`Tool`，以及用于中间步骤的提示（例如ReAct代理的“思考/动作/观察”追踪），代理执行的输出要么是要采取的下一个动作（`AgentAction`），要么是发送给用户的最终响应（`AgentFinish`）。
* `AgentExecutor`：是Agent的运行时，它实际上调用`Agent`，执行它选择的动作，将动作的输出传递回Agent，然后重复，直到达到`AgentFinish`。

> ❗ 准备您的API密钥
>
> 确保您已经按照先决条件设置了开发环境，并拥有调用LLM服务的有效API密钥，这里以OpenAI为例。
>
> 请确保您已经从环境变量中加载了OpenAPI密钥以供使用，如下所示。

In [ ]:
# Example `.env` config file.
![[ -f .env ]] || echo "OPENAI_API_KEY==<your_zhipu_ai_api_key>" > .env

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

### 3.1 Tool: Python Function + Description

首先，让我们看一下LangChain现成提供的一些内置Tool。

In [31]:
!pip install numexpr -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 583.1 kB/s eta 0:00:00a 0:00:01


In [32]:
from langchain.chat_models import ChatOpenAI

# [1] Some tools rely on LLM during its execution
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)
from langchain.agents import load_tools
from langchain.agents.load_tools import get_all_tool_names

math_tools = load_tools(['llm-math'], llm=llm)  # [1] Tool for arithmetic calculation
meteo_tools = load_tools(['open-meteo-api'], llm=llm)  # [2] Tool for weather info
wiki_tools = load_tools(['wikipedia'])  # [3] Tool for searching on Wikipedia

# [4] Print total list of builtin tool names
print(get_all_tool_names())

['python_repl', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal', 'sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-serper', 'google-scholar', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api']


#### `llm_math`

In [33]:
llm_math = math_tools[0]

# [1] Try a simple equation.
print(f'LLM Math: 2 + 2 => {llm_math.run("What is 2 + 2?")}')

# [2] How about a slightly diffucult one? Recall that pure LLM may fail on this example.
print(f'LLM Math: (4829 + 2930) * 1923 => {llm_math.run("Sum 4829 and 2930, and then multiply by 1923.")}')

# [3] Pure LLM failed to reach the correct answer.
print(f'Pure LLM: \n{llm.predict("Sum 4829 and 2930, and then multiply by 1923.")}')

LLM Math: 2 + 2 => Answer: 4
LLM Math: (4829 + 2930) * 1923 => Answer: 14920557
Pure LLM: 
The sum of 4829 and 2930 is 7759. 

To find the result after multiplying by 1923, we would multiply 7759 by 1923.

7759 multiplied by 1923 equals 14,930,157.


#### `open-meteo-api`

In [34]:
meteo = meteo_tools[0]
print(meteo.run("What's the weather in Paris?"))

The current weather in Paris is 8.2°C with a windspeed of 14.6 km/h coming from the northwest direction. The weather condition is identified by the WMO code 61.


In [35]:
from langchain.agents import tool
from datetime import date

@tool  # [1] We use the `tool` decorator to create new `Tool` instance
def time(text: str) -> str:
    # [2] The docstring (wrapped in """ """) are used as tool description
    #     (which is sent to LLM when used by agent)
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())  # [3] The actual logic for this `Tool`, i.e, return today's date

In [36]:
time.run('')  # Note the input is not used in our customed `Tool`

'2023-11-24'

另一个自定义工具，它接受多个参数作为输入并返回一个单一的字符串。

In [37]:
from typing import Optional

from langchain.tools import tool
import requests

@tool
def post_message(url: str, body: dict, parameters: Optional[dict] = None) -> str:
    """Sends a POST request to the given url with the given body and parameters."""
    result = requests.post(url, json=body, params=parameters)
    return f"Status: {result.status_code} - {result.text}"

### 3.2  Agent: Chain Equipped with Tools

LangChain已经定义了一些内置的Agent类型，我们可以直接在其基础上构建我们的应用程序。

In [38]:
from langchain.agents.types import AgentType
print([item.name for item in AgentType])

['ZERO_SHOT_REACT_DESCRIPTION', 'REACT_DOCSTORE', 'SELF_ASK_WITH_SEARCH', 'CONVERSATIONAL_REACT_DESCRIPTION', 'CHAT_ZERO_SHOT_REACT_DESCRIPTION', 'CHAT_CONVERSATIONAL_REACT_DESCRIPTION', 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION', 'OPENAI_FUNCTIONS', 'OPENAI_MULTI_FUNCTIONS']


In [ ]:
让我们看一个例子，即ZERO_SHOT_REACT_DESCRIPTION，它类似于零-shot ReAct风格的Agent。

In [39]:
from langchain.agents import load_tools
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)
tools = load_tools(['llm-math', 'open-meteo-api'], llm=llm)

agent = ZeroShotAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
)

请注意，LangChain中的`Agent`本身不运行，相反，它定义了适当的LLM、工具和提示，在`AgentExecutor`中执行时使用。让我们看看`ZeroShotAgent`是如何构建其提示的。

In [ ]:
print(agent.llm_chain.prompt.template)

注意，`{input}` 定义了用户输入或问题的位置，例如，“哪支球队赢得了2022年的FIFA世界杯？”；`{agent_scratchpad}` 是代理呈现其进一步执行的中间步骤的位置，例如，ReAct代理的“思考/动作/观察”三元组序列。按设计，在LangChain中，每个`Agent`都应该在其提示模板中定义一个变量`{agent_scratchpad}`。

### 3.3 AgentExecutor: Where Agents Execute

`AgentExecutor`是`Agent`（就像我们上面定义的那样）实际执行的地方。根据我们希望代理运行的方式，可以有不同类型的`AgentExecutor`。大多数情况下，我们希望使用LangChain提供的默认`AgentExecutor`。

以下代码片段来自`AgentExecutor`，展示了LangChain中通常如何执行`Agent`。
```python
class AgentExecutor(Chain):
    ...
    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """Run text through and get agent response."""
        ...
        # [1] To prevent `Agent`s from running into an infinite loop, `AgentExecutor` use
        #     both number of LLM invocations (`iterations`) and used time (`time_elapsed`)
        #     to stop execution even if `Agent` do not want to finish
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # [2] We now enter into the agent loop (until it returns something).
        while self._should_continue(iterations, time_elapsed):
            # [3] Take a single step in the "Thought/Action/Observation" loop, 
            #     return either `AgentAction` plus input or `AgentFinish`
            next_step_output = self._take_next_step(...)
            if isinstance(next_step_output, AgentFinish):  # [4] Return if LLM decides to finish
                return self._return(
                    next_step_output, intermediate_steps, run_manager=run_manager
                )
    
            intermediate_steps.extend(next_step_output)  # [5] Store current step, i.e, `AgentAction` plus input
            if len(next_step_output) == 1:
                next_step_action = next_step_output[0]
                # See if tool should return directly
                tool_return = self._get_tool_return(next_step_action)
                if tool_return is not None:  # [6] Check the next `AgentAction` wants to return directly
                    return self._return(
                        tool_return, intermediate_steps, run_manager=run_manager
                    )
            iterations += 1
            time_elapsed = time.time() - start_time
        # [7] Deal with early stop, can still return something even if stopped in the middle
        output = self.agent.return_stopped_response(
            self.early_stopping_method, intermediate_steps, **inputs
        )
        return self._return(output, intermediate_steps, run_manager=run_manager)
    ...
```

### 3.4 Put It Together

现在让我们将`Tool`、`Agent`和`AgentExecutor`结合起来，看看LangChain代理有哪些功能。

In [41]:
from langchain.agents import load_tools, AgentExecutor
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.7)
tools = load_tools(['llm-math', 'open-meteo-api'], llm=llm)

agent = ZeroShotAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
)

executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
)

print(executor.invoke('What is the weather in Berlin? Raise it to the power of 2.'))

{'input': 'What is the weather in Berlin? Raise it to the power of 2.', 'output': 'The weather in Berlin is currently 7.2°C and when raised to the power of 2, it is 51.84.'}
